**H2-Companion** 

Zuerst soll man die richtige Packages und Libraries importieren: 

In [1]:
import math
import numpy as np
import numpy_financial as npf
from scipy.optimize import brentq as root
from rhodium import *
import random
import pandas as pd
import time
import sys
import matplotlib.pyplot as plt
from j3 import J3

Die Cash Flow Funktion kann jetzt implementiert werden. Hier werden die folgenden Gleichungen erfüllt: 

**Variablenliste:**

-year $$\text{Jahr: } ano \space [Jahr]$$
-electricity_price $$\text{Strompreis: } C_{strom} \space [EURO/kWh]$$
-effizienz_elektrolyseur $$\text{Wirkungsgrad Elektrolyseur:}  \space \eta \space [kWh/kg]$$
-C_H2 $$\text{Preis Wasserstoff: } C_{H_2} \space  [EURO/kg]$$
-betriebsStunden $$\text{Betriebsstunden: } t_{betrieb} \space [h/ano]$$

**Gleichungen:** 
$$\text{Kosten Elektrolyseur: } \space C_{elek} =t_{betrieb} \cdot ano \cdot C_{strom} \space [EURO/kW] $$
$$\text{Ertrag Elektrolyseur: } \space E_{elek} = \frac{C_{H_2} \cdot t_{betrieb} \cdot ano}{\eta} \space [EURO/kW] $$
$$\text{Umsatz: } \space CF_{elek} =E_{elek}-C_{elek} \space [EURO/kW] $$

In [2]:
def calculate_cash_flow(year: int = 20,  # Jahre im Betrieb-Projectzeit [a] (fest... glaube ich)
                        electricity_price: float = 0.07,  # Einheit [EURO/kWh]
                        effizienz_elektrolyseur: float = 0.55,  # Einheit [kWh/kg]
                        C_H2: float = 5,  # Einheit [EURO/kg]
                        betriebsStunden: int = 5000  # Einheit [h/a]
                        ) -> float:
    Kosten_Elektrolyseur = betriebsStunden * year * electricity_price
    Ertrag_Eletrolyseur = (C_H2 * (betriebsStunden * year)) / effizienz_elektrolyseur
    net_win_elektrolyseur = Ertrag_Eletrolyseur - Kosten_Elektrolyseur # = Umsatz
    return net_win_elektrolyseur #Umsatz 

**Model für Rhodium:**

Hier wird das Model gebildet, sodass Rhodium die Funktion (siehe oben) bearbeiten kann.

Als "Antwort" von der Funktion: Umsatz $ CF_{elek} $

Kontrollierbarkeit: Betriebsstunden $ t_{betrieb} $

Unsicherheiten: $ C_{strom}, \, C_{H_2}, \, \eta $


In [3]:
def return_model() -> Model:
    model = Model(calculate_cash_flow)
    model.parameters = [
        Parameter("electricity_price"),
        Parameter("effizienz_elektrolyseur"),
        Parameter("C_H2"),
        Parameter("betriebsStunden")
    ]

    model.responses = [
        Response("net_win_elektrolyseur", Response.MAXIMIZE)
    ]

    model.levers = [
        RealLever("betriebsStunden", 3000, 5000),
    ]

    model.uncertainties = [
        UniformUncertainty("electricity_price", 0.03, 0.10),
        UniformUncertainty("effizienz_elektrolyseur", 0.35, 0.60),
        UniformUncertainty("C_H2", 3, 8)
    ]

    return model

**Senstivity Analysis!**

Hier werden drei Methoden benutzt. "Delta-sa" und "FAST-sa"

In [20]:
def main():
    # Conduct analysis
    model = return_model()

    # Convert odict_keys to list explicitly for parameters and responses
    model_parameters = list(model.parameters.keys())
    model_responses = list(model.responses.keys())

    # Print the model's parameters and responses for debugging
    print("Model Parameters:", model_parameters)
    print("Model Responses:", model_responses)

    policies = optimize(model, "NSGAII", 1000)
    # Print the optimized policies
    for policy in policies:
        print(policy)

    result_fast = sa(model, "net_win_elektrolyseur", method="fast", nsamples=1000)
    print("Fourier Amplitude Sensitivity Test (FAST)")
    print("------------------------------------------------------------")
    print(result_fast)
    print()
    print()

    result_delta = sa(model, "net_win_elektrolyseur", method="delta", nsamples=1000)
    print("Delta Sensitivity Test")
    print("------------------------------------------------------------")
    print(result_delta)
    print()
    print()
    
    #result_ff = sa(model, "net_win_elektrolyseur", method="ff", nsamples=1000)
    #print("Fractional Factorial Sensitivity Analysis")
    #print("------------------------------------------------------------")
    #print(result_ff)
    #print()
    
    result_dgsm = sa(model, "net_win_elektrolyseur", method="dgsm", nsamples=1000)
    print("Derivative-based Global Sensitivity Measure")
    print("------------------------------------------------------------")
    print(result_dgsm)
    print()
    print()

    #result_morris = sa(model, "net_win_elektrolyseur", method="morris", nsamples=1000)
    #print("Method of Morris")
    #print("------------------------------------------------------------")
    #print(result_morris)
    #print()

    #result_sobol = sa(model, "net_win_elektrolyseur", method="sobol", nsamples=1000)
    #print("Method of Sobol")
    #print("------------------------------------------------------------")
    #print(result_sobol)
    #print()

    output = cache("output", lambda: optimize(model, "NSGAII", 10000))
    output.save('optimization_results.csv')

if __name__ == "__main__":
    main()


Model Parameters: ['electricity_price', 'effizienz_elektrolyseur', 'C_H2', 'betriebsStunden']
Model Responses: ['net_win_elektrolyseur']
OrderedDict([('betriebsStunden', 4999.999967515782), ('net_win_elektrolyseur', 902090.9032301655)])
Fourier Amplitude Sensitivity Test (FAST)
------------------------------------------------------------
First order sensitivity indices (confidence interval):
          electricity_price: +0.000032 (+0.047028)
    effizienz_elektrolyseur: +0.256464 (+0.043074)
                       C_H2: +0.723125 (+0.041992)
Total order sensitivity indices (confidence interval):
          electricity_price: +0.001789 (+0.070931)
    effizienz_elektrolyseur: +0.276623 (+0.065190)
                       C_H2: +0.743552 (+0.068232)


Delta Sensitivity Test
------------------------------------------------------------
First order sensitivity indices (confidence interval):
          electricity_price: +0.005663 (+0.009868)
    effizienz_elektrolyseur: +0.231696 (+0.050420)
 


### Interpretation der Daten

#### Fourier Amplitude Sensitivity Test (FAST)

**First Order Sensitivity Indices (Si):**
- **electricity_price:** +0.000032 (+0.047028)
- **effizienz_elektrolyseur:** +0.256464 (+0.043074)
- **C_H2:** +0.723125 (+0.041992)

**Total Order Sensitivity Indices (STi):**
- **electricity_price:** +0.001789 (+0.070931)
- **effizienz_elektrolyseur:** +0.276623 (+0.065190)
- **C_H2:** +0.743552 (+0.068232)

**Interpretation:**
- Der **electricity_price** hat sowohl in den ersten als auch in den Gesamt-Sensitivitätsindizes einen sehr geringen Einfluss auf das Modelloutput, was durch die niedrigen Werte angezeigt wird. Der First Order Index liegt bei 0.000032 und der Total Order Index bei 0.001789.
- **effizienz_elektrolyseur** hat einen moderaten Einfluss, wobei der First Order Index 0.256464 und der Total Order Index 0.276623 beträgt. Dies bedeutet, dass etwa 25-27% der Varianz im Modelloutput durch diesen Parameter erklärt werden.
- **C_H2** (Preis Wasserstoff) hat den größten Einfluss auf das Modelloutput. Der First Order Index beträgt 0.723125 und der Total Order Index 0.743552. Das bedeutet, dass dieser Parameter allein über 72% der Varianz im Modelloutput erklärt.

#### Delta Sensitivity Test

**First Order Sensitivity Indices (Si):**
- **electricity_price:** +0.005663 (+0.009868)
- **effizienz_elektrolyseur:** +0.231696 (+0.050420)
- **C_H2:** +0.716355 (+0.025224)

**Borgonovo's Delta Moment (Delta):**
- **electricity_price:** +0.060987 (+0.017155)
- **effizienz_elektrolyseur:** +0.132286 (+0.021586)
- **C_H2:** +0.434467 (+0.018781)

**Interpretation:**
- Die Delta-Sensitivitätsindizes bestätigen die Ergebnisse des FAST. Der **electricity_price** hat nur einen geringen Einfluss, während **effizienz_elektrolyseur** und **C_H2** einen größeren Einfluss haben.
- Borgonovo's Delta Moment gibt zusätzliche Einblicke in die Bedeutung der Parameter. **C_H2** ist der dominierende Parameter mit dem höchsten Delta-Wert, was auf seine große Bedeutung für das Modelloutput hinweist.

#### Derivative-based Global Sensitivity Measure (DGSM)

**DGSM's Importance Criteria (stdev):**
- **electricity_price:** +10000000000.000048 (+0.001100)
- **effizienz_elektrolyseur:** +8098439234403.797852 (+6998713284374.288086)
- **C_H2:** +47739557850.565468 (+15027988589.562323)

**DGSM's Sensitivity Index (G):**
- **electricity_price:** +0.007496 (+0.000131)
- **effizienz_elektrolyseur:** +6.070510 (+893933732624.045288)
- **C_H2:** +0.035785 (+2044343164.610078)

**Interpretation:**
- Die DGSM-Indizes zeigen, dass der **effizienz_elektrolyseur** (Wirkungsgrad des Elektrolyseurs) den größten Einfluss auf die Modellvarianz hat, gefolgt von **C_H2**. Der hohe Wert für den **effizienz_elektrolyseur** deutet darauf hin, dass kleine Änderungen in diesem Parameter zu großen Änderungen im Modelloutput führen können.
- Der **electricity_price** hat trotz eines hohen Importance Criteria einen sehr niedrigen Sensitivity Index, was darauf hinweist, dass seine Bedeutung im Modell gering ist.